In [2]:
pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095501 sha256=8b8f15e2a87dec854e0d8da1cf6e5042570fbe6c1db0b70ac1b3917bf8233032
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNWithMeans, KNNBaseline, accuracy
from surprise.model_selection import train_test_split


In [29]:
data = Dataset.load_builtin('ml-100k')
data_df = pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
anti_set = data.build_full_trainset().build_anti_testset() # funkcja tworzy zbiór par użytkownika i filmu których nie ma w oryginalnym zbiorze

In [30]:
all_movies = data_df['item_id'].drop_duplicates()
users = data_df['user_id'].drop_duplicates()

In [31]:
trainset, testset = train_test_split(data, test_size=0.25)

In [95]:
knn = KNNWithMeans(k=40, sim_options ={'name': 'pearson', 'user_based': True})

In [96]:
knn.fit(trainset)
predictions_knn = knn.test(testset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [97]:
pred_df = pd.DataFrame(predictions_knn).merge(data_df , left_on = ['uid', 'iid'], right_on = ['user_id', 'item_id'])

pred_df

,uid,iid,r_ui,est,details,user_id,item_id,rating,timestamp
0,59,622,4.0,3.525652,"{'actual_k': 27, 'was_impossible': False}",59,622,4.0,888206015
1,677,1,4.0,3.774703,"{'actual_k': 40, 'was_impossible': False}",677,1,4.0,889399229
2,267,17,4.0,3.719673,"{'actual_k': 40, 'was_impossible': False}",267,17,4.0,878971773
3,642,1146,1.0,3.187747,"{'actual_k': 1, 'was_impossible': False}",642,1146,1.0,886570084
4,405,453,3.0,1.000000,"{'actual_k': 8, 'was_impossible': False}",405,453,3.0,885548385
...,...,...,...,...,...,...,...,...,...
24995,322,656,5.0,4.733367,"{'actual_k': 16, 'was_impossible': False}",322,656,5.0,887314027
24996,456,232,2.0,3.311623,"{'actual_k': 40, 'was_impossible': False}",456,232,2.0,881374389
24997,716,724,4.0,3.928315,"{'actual_k': 40, 'was_impossible': False}",716,724,4.0,879796138
24998,393,41,4.0,2.853917,"{'actual_k': 22, 'was_impossible': False}",393,41,4.0,889728736


In [98]:
anti_pred = knn.test(anti_set)
pred_df_anti = pd.DataFrame(anti_pred).merge(all_movies , left_on = ['iid'], right_on = ['item_id'])
pred_df_anti = pd.DataFrame(pred_df_anti).merge(users , left_on = ['uid'], right_on = ['user_id'])
pred_df_anti

,uid,iid,r_ui,est,details,item_id,user_id
0,196,302,3.52986,3.758052,"{'actual_k': 40, 'was_impossible': False}",302,196
1,196,377,3.52986,1.673911,"{'actual_k': 3, 'was_impossible': False}",377,196
2,196,51,3.52986,2.808753,"{'actual_k': 20, 'was_impossible': False}",51,196
3,196,346,3.52986,3.468617,"{'actual_k': 29, 'was_impossible': False}",346,196
4,196,474,3.52986,3.956154,"{'actual_k': 40, 'was_impossible': False}",474,196
...,...,...,...,...,...,...,...
1486121,13,1007,3.52986,3.606448,"{'actual_k': 33, 'was_impossible': False}",1007,13
1486122,13,1118,3.52986,2.857975,"{'actual_k': 34, 'was_impossible': False}",1118,13
1486123,13,108,3.52986,2.932195,"{'actual_k': 40, 'was_impossible': False}",108,13
1486124,13,1241,3.52986,1.847262,"{'actual_k': 5, 'was_impossible': False}",1241,13


In [99]:
def recommend(user, n_movies):
  return pred_df_anti[pred_df_anti['uid']==str(user)].sort_values('est',ascending=False).head(n_movies)

In [100]:
#zadanie 3
recommend(244,10)

,uid,iid,r_ui,est,details,item_id,user_id
4546,244,1643,3.52986,5.000000,"{'actual_k': 2, 'was_impossible': False}",1643,244
4235,244,1536,3.52986,5.000000,"{'actual_k': 1, 'was_impossible': False}",1536,244
4091,244,814,3.52986,5.000000,"{'actual_k': 1, 'was_impossible': False}",814,244
4078,244,119,3.52986,5.000000,"{'actual_k': 2, 'was_impossible': False}",119,244
4333,244,1591,3.52986,5.000000,"{'actual_k': 1, 'was_impossible': False}",1591,244
4500,244,1599,3.52986,5.000000,"{'actual_k': 1, 'was_impossible': False}",1599,244
4223,244,1388,3.52986,4.995482,"{'actual_k': 4, 'was_impossible': False}",1388,244
4509,244,1642,3.52986,4.980748,"{'actual_k': 2, 'was_impossible': False}",1642,244
4170,244,1175,3.52986,4.815432,"{'actual_k': 2, 'was_impossible': False}",1175,244
4261,244,1398,3.52986,4.813066,"{'actual_k': 2, 'was_impossible': False}",1398,244


In [58]:
#zadanie 1
data_df[data_df['user_id']=='122']

,user_id,item_id,rating,timestamp
16,122,387,5.0,879270459
726,122,509,4.0,879270511
1229,122,378,4.0,879270769
1247,122,510,4.0,879270327
2603,122,215,4.0,879270676
...,...,...,...,...
82310,122,519,4.0,879270129
92590,122,1074,4.0,879270901
95100,122,197,5.0,879270482
99241,122,699,5.0,879270541


In [62]:
#zadanie 2
knnB = KNNBaseline(k=20, sim_options ={'name': 'pearson', 'user_based': True})
knnB.fit(trainset)
predictions_knnB = knnB.test(testset)
pred_df_knnB = pd.DataFrame(predictions_knnB).merge(data_df , left_on = ['uid', 'iid'], right_on = ['user_id', 'item_id'])
anti_pred_knnB = knnB.test(anti_set)
pred_df_anti_knnB = pd.DataFrame(anti_pred_knnB).merge(all_movies , left_on = ['iid'], right_on = ['item_id'])
pred_df_anti_knnB = pd.DataFrame(pred_df_anti_knnB).merge(users , left_on = ['uid'], right_on = ['user_id'])

pred_df_anti_knnB

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


,uid,iid,r_ui,est,details,item_id,user_id
0,196,302,3.52986,3.703401,"{'actual_k': 20, 'was_impossible': False}",302,196
1,196,377,3.52986,1.697523,"{'actual_k': 3, 'was_impossible': False}",377,196
2,196,51,3.52986,2.818523,"{'actual_k': 20, 'was_impossible': False}",51,196
3,196,346,3.52986,3.546839,"{'actual_k': 20, 'was_impossible': False}",346,196
4,196,474,3.52986,4.242576,"{'actual_k': 20, 'was_impossible': False}",474,196
...,...,...,...,...,...,...,...
1486121,13,1007,3.52986,3.802734,"{'actual_k': 20, 'was_impossible': False}",1007,13
1486122,13,1118,3.52986,2.804188,"{'actual_k': 20, 'was_impossible': False}",1118,13
1486123,13,108,3.52986,3.125331,"{'actual_k': 20, 'was_impossible': False}",108,13
1486124,13,1241,3.52986,1.835330,"{'actual_k': 5, 'was_impossible': False}",1241,13


In [94]:
knnB.test([['244','50',None]])


[Prediction(uid='244', iid='50', r_ui=None, est=4.388046397002306, details={'actual_k': 20, 'was_impossible': False})]